Notebook Entrainement de modèle depuis les données mongoDB vers MLFLOW

In [ ]:
import os
import json
import pymongo
import mlflow
import mlflow.sklearn
import numpy as np
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder


In [ ]:
# Connexion à MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")  
db = client.get_database("dev")
collection = db.get_collection("raw_data")

In [ ]:
# Charger les données depuis MongoDB
def load_data_from_mongo():
    data = list(collection.find())
    texts = [item.get("readme_clean", "") for item in data]
    labels = [item.get("mainLanguage", "") for item in data]
    return texts, labels


In [ ]:
# Charger et préparer les données
texts, labels = load_data_from_mongo()
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Tokenisation des textes
tokenized_texts = [word_tokenize(text) for text in texts]



In [ ]:
# Entraînement du modèle Word2Vec
word2vec_model = Word2Vec(sentences=tokenized_texts, vector_size=100, window=5, min_count=1, workers=4)



In [ ]:
# Fonction pour vectoriser les documents
def vectorize_documents(tokenized_texts, model):
    document_vectors = []
    for tokens in tokenized_texts:
        vectors = [model.wv[word] for word in tokens if word in model.wv]
        if vectors:
            document_vectors.append(np.mean(vectors, axis=0))
        else:
            document_vectors.append(np.zeros(model.vector_size))
    return document_vectors

document_vectors = vectorize_documents(tokenized_texts, word2vec_model)



In [ ]:
# Division des données en train/test
X_train, X_test, y_train, y_test = train_test_split(document_vectors, encoded_labels, test_size=0.2, random_state=42)



In [ ]:
# Entraînement du classifieur
classifier = RandomForestClassifier(n_estimators=100, random_state=42)
classifier.fit(X_train, y_train)

In [ ]:
# Configuration de MLflow
mlflow.set_tracking_uri("http://localhost:8090")

# Enregistrement du modèle dans MLflow
with mlflow.start_run():
    mlflow.sklearn.log_model(classifier, "random_forest_model")
    mlflow.log_param("model_type", "RandomForestClassifier")

print("Modèle entraîné et enregistré dans MLflow !")